In [2]:
import pandas as pd

In [13]:
logs = pd.DataFrame({
    "id":  [1,2,3,4,5,6,7, 8],
    "num": [1,1,1,2,1,2,2, 2]
})

logs

,id,num
0,1,1
1,2,1
2,3,1
3,4,2
4,5,1
5,6,2
6,7,2
7,8,2


In [14]:
len(logs.num)

8

In [15]:
repeated_3 = []
for i in range(len(logs.num)-2):
    if logs.num[i] == logs.num[i+1] == logs.num[i+2]:
        repeated_3.append(i)

In [24]:
repeated_3

[0, 5]

In [34]:
new = logs.iloc[repeated_3]
new

,id,num
0,1,1
5,6,2


In [36]:
new['num']

0    1
5    2
Name: num, dtype: int64

In [40]:
df = new.rename(columns = {'num':'ConsecutiveNums'})
print(df)

   id  ConsecutiveNums
0   1                1
5   6                2


In [35]:
table = logs.num[repeated_3]